# Examples
Here we provide some examples with real-world data to examplify the difference between the different approaches.

## Breast Cancer Prediction
In this example, we look at the Breast Cancer dataset from scikit-learn. The classification task is to predict whether the subject has cancer or not.

In [5]:
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Import our custom functions
from src.metrics import create_metrics
from src.cv import run_cv

In [6]:
X, y = load_breast_cancer(return_X_y=True)

# print dataset shape
print(f"Dataset shape: {X.shape}")
# Check class balance
print(f"Class distribution:\n{sum(y)} positive, {len(y) - sum(y)} negative, ratio: {sum(y) / len(y):.2f}")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Dataset shape: (569, 30)
Class distribution:
357 positive, 212 negative, ratio: 0.63


In [7]:
model = LogisticRegression()
metrics = create_metrics(["accuracy", "auc"])
results = run_cv(model, X_scaled, y, metrics, n_splits=5, stratified=True, random_state=1)

results_df = pd.DataFrame(results)
print(results_df)

           average    pooled
accuracy  0.978910  0.978910
auc       0.995428  0.995098


## Cognitive Impairment
In this example, we look at the Oasis3 data. We have cortical volume for various subjects and sessions that are cognitively normal or impaired. The classification task is to predict whether the subject is cogntiviely normal or not, based on cortical features.

In [8]:
# Load the dataset
data_path = Path("../data/oasis3_fs_mci.tsv")
df = pd.read_csv(data_path, sep="\t")

print(f"Dataset shape: {df.shape}")

# drop rows that have empty cells / NAs
df = df.dropna(axis=0, how="any")

# only keep first occurence of each subject
df_baseline = df.drop_duplicates(subset=["subject"], keep="first")
print(f"Shape of baseline data: {df_baseline.shape}")

Dataset shape: (2832, 104)
Shape of baseline data: (1029, 104)


In [11]:
# split into X
X = df_baseline.drop(columns=["subject", "session", "age", "cognitiveyly_normal"])
X = X.apply(pd.to_numeric, errors="coerce")
X = X.to_numpy()

# standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# and y
y = df_baseline["cognitiveyly_normal"].to_numpy()
y_binary = (y == True).astype(int)

# Check class balance
print(f"Class distribution:\n{sum(y_binary)} positive, {len(y_binary) - sum(y_binary)} negative, ratio: {sum(y_binary) / len(y_binary):.2f}")

Class distribution:
755 positive, 274 negative, ratio: 0.73


In [ ]:
## Run cross-validation
model = LogisticRegression(max_iter=1000000)
metrics = create_metrics(["accuracy", "rocauc"])
results = run_cv(model, X_scaled, y_binary, metrics, n_splits=5, stratified=True, random_state=1)
results_df = pd.DataFrame(results)
print(results_df)

           average    pooled
accuracy  0.820214  0.820214
auc       0.822567  0.822845
